<a href="https://colab.research.google.com/github/Domdoug/Regioes-de-Saude/blob/master/Regioes_de_Saude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
nome_colunas = ['CD_MUNICIPIO', 'SG_UF', 'NM_REGIAO_SAUDE_ATUAL', 'NM_MUNICIPIO',
       'NM_REGIAO_SAUDE_ANTERIOR', 'NM_MUNICIPIOS_IN_37', 'ESTADO', 'REGIAO_PAIS']

In [0]:
df_regioes = pd.read_excel('BI_Regionais de saude.xlsx', sheet_name='Dados', skiprows=1, usecols="A:H", names=nome_colunas)

In [4]:
df_regioes.sort_values(by='SG_UF', ascending=True).head()

,CD_MUNICIPIO,SG_UF,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS
1483,120001,AC,Baixo Acre e Purus,Acrelândia,Acre,Acrelândia,ACRE,NORTE
1212,120005,AC,Alto Acre,Assis Brasil,Brasiléia,Assis Brasil,ACRE,NORTE
3009,120030,AC,Juruá e Tarauacá/Envira,Feijó,Cruzeiro do Sul,Feijó,ACRE,NORTE
1211,120070,AC,Alto Acre,Xapuri,Brasiléia,Xapuri,ACRE,NORTE
3010,120033,AC,Juruá e Tarauacá/Envira,Mâncio Lima,Cruzeiro do Sul,Mâncio Lima,ACRE,NORTE


In [5]:
df_regioes.shape

(5571, 8)

In [0]:
# INICIO DO PROCESSAMENTO

In [0]:
# Esta linha e para agilizar o groupby. Talvez desnecessaria. Lógica do do SAS
df_regioes.sort_values(by=['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'], inplace=True)

In [0]:
# Aplicar na coluna CD_MUNICIPIO uma funcao para colocar os codigos do municipios em formato de conjunto para comparar,
# agrupados por regiao atual e UF, com o agrupamento dos municipios correspondentes da regiao anterior

lista1 = df_regioes.groupby(['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'])['CD_MUNICIPIO'].apply(lambda x:list(set(x)))

In [0]:
# Agora a lista para a Região de Saúde anterior
df_regioes.sort_values(by=['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'], inplace=True)

In [0]:
# Agora o código para a lista2: Regiao de Saude Anterior
lista2 = df_regioes.groupby(['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'])['CD_MUNICIPIO'].apply(lambda x:list(set(x)))

In [0]:
# Para transformar o formato agrupado em DataFrame
lista1 = lista1.reset_index()
lista2 = lista2.reset_index()

In [11]:
lista1.head()

,NM_REGIAO_SAUDE_ATUAL,SG_UF,CD_MUNICIPIO
0,10ª RS Cascavel,PR,"[410754, 410500, 410630, 411275, 411670, 41053..."
1,10ª Região,PB,"[250915, 251620, 251398, 251420, 251720, 25084..."
2,10ª Região Limoeiro do Norte,CE,"[230690, 230600, 230760, 231080, 231310, 23115..."
3,10ª Região de Saúde,AL,"[270240, 270330, 270500, 270642, 270580, 27071..."
4,11ª RS Campo Mourão,PR,"[412800, 410755, 411400, 410250, 410770, 41039..."


In [12]:
lista2.head()

,NM_REGIAO_SAUDE_ANTERIOR,SG_UF,CD_MUNICIPIO
0,Acaraú,CE,"[230020, 230725, 230890, 230230, 230425, 23078..."
1,Acre,AC,"[120001, 120034, 120038, 120040, 120043, 12004..."
2,Afogados da Ingazeira,PE,"[261440, 261153, 260770, 261280, 260390, 26071..."
3,Alagoinhas,BA,"[291650, 291590, 292970, 291370, 291050, 29022..."
4,Alfenas,MG,"[310530, 310410, 314510, 311440, 310160, 31669..."


In [13]:
lista1.shape

(438, 3)

In [14]:
lista2.shape

(358, 3)

In [0]:
# ================ TESTES=========================

In [0]:
# verificar se tem regiões que abrangem mais de uma UF:
# df_regioes.groupby(['NM_REGIAO_SAUDE_ATUAL', 'SG_UF']).size()
df_unicos = df_regioes.drop_duplicates(subset=['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'])

In [16]:
df_unicos.shape  #438, 8

(438, 8)

In [0]:
# Agrupa para selecionar Região de Saúde no DataFrame. Depois aplica o o metodo duplicated, para selecionar os efetivamente repetidos
linhas_duplicadas = df_regioes.groupby(['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'], as_index=False).count()

In [18]:
linhas_duplicadas.shape

(438, 8)

In [19]:
# Seleciona todas as linhas duplicadas baseadas em uma lista de colunas. O parametro keep é para exibir todos os repetidos e não o último ou primeiro
linhas_duplicadas = linhas_duplicadas[linhas_duplicadas.duplicated(subset=['NM_REGIAO_SAUDE_ATUAL'], keep=False)]
linhas_duplicadas.shape  #(17, 8)
linhas_duplicadas.head()
# Análise: Observando o campo NM_REGIAO_SAUDE_ATUAL e SG_UF, observa-se que: As regiões repetidas e que são de UF's contíguas São: Baixo Amazonas (AM e PA).
# O restante não são contíguas. Destaque para a Região de Saúde nomeada de "NORTE". Observando-se RJ e ES, que são contíguos, porém as respectivas Regiões situam-se no Norte de cada Estado.


,NM_REGIAO_SAUDE_ATUAL,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS
115,Baixo Amazonas,AM,5,5,5,5,5,5
116,Baixo Amazonas,PA,14,14,13,13,14,14
142,Central,ES,18,18,18,18,18,18
143,Central,GO,26,26,26,26,26,26
144,Central,RO,14,14,14,14,14,14


In [0]:
# Até aqui, para a região Norte, não tem Problemas
teste = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Norte']
# Para o caso da Região Baixo Amazonas que abrange 2 UF's, deve-se colocar a condição de agregar, para este caso, as duas UF's 
# teste = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']

In [0]:
# teste.to_excel('teste2.xlsx')
lista1.to_excel('lista1.xlsx')

In [21]:
teste.shape

(3, 3)

In [22]:
teste.head()

,NM_REGIAO_SAUDE_ATUAL,SG_UF,CD_MUNICIPIO
271,Norte,ES,"[320160, 320515, 320100, 320390, 320360, 32042..."
272,Norte,GO,"[521377, 521410, 522145, 522020, 520357, 52196..."
273,Norte,RJ,"[330240, 330500, 330475, 330093, 330415, 33048..."


In [23]:
teste['CD_MUNICIPIO'].str.len()

271    14
272    13
273     8
Name: CD_MUNICIPIO, dtype: int64

In [0]:
teste = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']

In [25]:
teste['CD_MUNICIPIO'].str.len()

115     5
116    14
Name: CD_MUNICIPIO, dtype: int64

In [0]:
# verificar se tem regiões (ANTERIOR que abrangem mais de uma UF:
# df_regioes.groupby(['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF']).size()
df_unicos = df_regioes.drop_duplicates(subset=['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'])

In [27]:
df_unicos.shape  #362, 8

(362, 8)

In [28]:
# Agrupa para selecionar Região de Saúde no DataFrame. Depois aplica o o metodo duplicated, para selecionar os efetivamente repetidos
linhas_duplicadas = df_regioes.groupby(['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'], as_index=False).count()
linhas_duplicadas.shape # 358, 8

(358, 8)

In [29]:
# Seleciona todas as linhas duplicadas baseadas em uma lista de colunas. O parametro keep é para exibir todos os repetidos e não o último ou primeiro
linhas_duplicadas = linhas_duplicadas[linhas_duplicadas.duplicated(subset=['NM_REGIAO_SAUDE_ANTERIOR'], keep=False)]
linhas_duplicadas.shape  #(17, 8)
linhas_duplicadas
# Análise: Observando o campo NM_REGIAO_SAUDE_ANTERIOR e SG_UF, observa-se que: As regiões repetidas e que são de UF's contíguas São: Baixo Amazonas (AM e PA).
# O restante não são contíguas.


,NM_REGIAO_SAUDE_ANTERIOR,SG_UF,CD_MUNICIPIO,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS
26,Baixo Amazonas,AM,5,5,5,5,5,5
27,Baixo Amazonas,PA,9,9,9,9,9,9
187,Metropolitana,PA,5,5,5,5,5,5
188,Metropolitana,PR,29,29,29,29,29,29
211,Norte,GO,13,13,13,13,13,13
212,Norte,RJ,8,8,8,8,8,8


In [0]:
# ================ FIM DOS TESTES =====================

In [0]:
# ========ROTINA INTERMEDIARIA================

# Rotina para agregar a regiao 'Baixo Amazonas' das UF's: AM e PA, por tratar-se de região interestadual. Caso único.
lista_aux1 = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']
lista_aux2 = lista2[lista2['NM_REGIAO_SAUDE_ANTERIOR']=='Baixo Amazonas']


In [31]:
lista_aux1.head()

,NM_REGIAO_SAUDE_ATUAL,SG_UF,CD_MUNICIPIO
115,Baixo Amazonas,AM,"[130050, 130340, 130290, 130068, 130300]"
116,Baixo Amazonas,PA,"[150145, 150050, 150530, 150565, 150600, 15047..."


In [32]:
lista_aux2.head()

,NM_REGIAO_SAUDE_ANTERIOR,SG_UF,CD_MUNICIPIO
26,Baixo Amazonas,AM,"[130050, 130340, 130290, 130068, 130300]"
27,Baixo Amazonas,PA,"[150145, 150050, 150565, 150600, 150285, 15048..."


In [0]:
# uso do drop para SG_UF  para facilitar o gropuby por NM_REGIAO_SAUDE_ATUAL
lista_aux1_new = lista_aux1.drop('SG_UF', axis=1)
lista_aux2_new = lista_aux2.drop('SG_UF', axis=1)

In [0]:
lista_aux1_fim = lista_aux1_new.groupby('NM_REGIAO_SAUDE_ATUAL')['CD_MUNICIPIO'].sum().reset_index()
lista_aux2_fim = lista_aux2_new.groupby('NM_REGIAO_SAUDE_ANTERIOR')['CD_MUNICIPIO'].sum().reset_index()

In [35]:
lista_aux1_fim.head()

,NM_REGIAO_SAUDE_ATUAL,CD_MUNICIPIO
0,Baixo Amazonas,"[130050, 130340, 130290, 130068, 130300, 15014..."


In [36]:
lista_aux2_fim.head()

,NM_REGIAO_SAUDE_ANTERIOR,CD_MUNICIPIO
0,Baixo Amazonas,"[130050, 130340, 130290, 130068, 130300, 15014..."


In [65]:
contar1 = list(lista_aux1_fim['CD_MUNICIPIO'])
# len(lista_aux2_fim['CD_MUNICIPIO'])
# Contar list of values within a pandas df
# contar1 = [sum(a) for a in zip(*lista_aux1_fim['CD_MUNICIPIO'])]
contar1

[[130050,
  130340,
  130290,
  130068,
  130300,
  150145,
  150050,
  150530,
  150565,
  150600,
  150475,
  150285,
  150510,
  150797,
  150480,
  150390,
  150040,
  150300,
  150680]]

In [66]:
len(contar1)

1

In [0]:
teste = lista1.copy()

In [0]:
#atualizacao somente da linha onde está o baixo amazonas. Não esquecer de deletar a outra linha. Baixo Amazonas (PA)
teste.loc[teste['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas', 'CD_MUNICIPIO'] = contar1

In [0]:
teste = teste.drop(teste[(teste['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas') & (teste['SG_UF']=='PA')].index) # nao esquecer index
# teste[teste['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']
# teste.head()

In [91]:
teste[teste['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']

,NM_REGIAO_SAUDE_ATUAL,SG_UF,CD_MUNICIPIO
115,Baixo Amazonas,AM,"[130050, 130340, 130290, 130068, 130300, 15014..."


In [0]:
# =========================================
# OPERAÇÕES DE CONCATENAÇÃO DAS LISTAS DE MUNICIPIOS EM LISTA POR REGIOES COM O DATAFRAME ORIGINAL

In [0]:
df_regioes_concat1 = pd.merge(df_regioes, lista1, how = 'left', left_on=['NM_REGIAO_SAUDE_ATUAL','SG_UF'], right_on=['NM_REGIAO_SAUDE_ATUAL','SG_UF'])
# Agora concatenação com a Regiao anterior, na otica da lista 2
df_regioes_concat2 = pd.merge(df_regioes, lista2, how = 'left', left_on=['NM_REGIAO_SAUDE_ANTERIOR','SG_UF'], right_on=['NM_REGIAO_SAUDE_ANTERIOR','SG_UF'])

In [0]:
df_regioes_concat1.shape

In [0]:
df_regioes_concat2.shape

In [0]:
df_regioes_concat1.head()

In [0]:
df_regioes_concat2.head()

In [0]:
# Renomear coluna que foi gerada na juncao do passo anterior
df_regioes_concat1.rename(columns={'CD_MUNICIPIO_y':'lista_regiao_atual'}, inplace=True)
df_regioes_concat2.rename(columns={'CD_MUNICIPIO_y':'lista_regiao_anterior'}, inplace=True)

In [0]:
# Verificar como os dataframes ficaram apos renomear coluna

In [0]:
df_regioes_concat1.head()

In [0]:
df_regioes_concat2.head()

In [0]:
# como são dois dataframes muito identicos em colunas, usei esta técnica para selecionar as do segundo dataframe que sejam diferentes
colunas_mantidas = df_regioes_concat2.columns.difference(df_regioes_concat1.columns)

In [0]:
colunas_mantidas

In [0]:
df_regioes_concat = pd.merge(df_regioes_concat1, df_regioes_concat2[colunas_mantidas], left_index=True, right_index=True, how='outer')

In [0]:
df_regioes_concat.rename(columns={'CD_MUNICIPIO_x':'CD_MUNICIPIO'}, inplace=True)

In [0]:
df_regioes_concat['flag_atual'] = np.where(df_regioes_concat['lista_regiao_atual'] == df_regioes_concat['lista_regiao_anterior'], 0, 1)

In [0]:
df_regioes_concat.head()

In [0]:
# ============ PASSO PARA MERGE COM A TABLE DE MUNICIPIOS PARA AGREGAR CAMPOS DESTA TABELA

In [0]:
df_municipios = pd.read_excel('AR_BR_RG_UF_MES_MIC_MUN_2018.xls')

In [0]:
df_municipios.head()

In [0]:
df_municipios.dropna(axis=0, how='any', subset=['CD_GCMUN'], inplace=True)

In [0]:
df_municipios['CD_MUNICIPIO_NDV'] = df_municipios['CD_GCMUN'].astype(str).str[:6]

In [0]:
type(df_municipios['CD_MUNICIPIO_NDV'][0])

In [0]:
df_regioes_concat['CD_MUNICIPIO'] = df_regioes_concat['CD_MUNICIPIO'].astype(str)

In [0]:
df_diferenca = pd.merge(df_regioes_concat, df_municipios, how = 'outer', left_on='CD_MUNICIPIO', right_on='CD_MUNICIPIO_NDV')

In [0]:
df_diferenca_qgis = df_diferenca[df_diferenca['flag_atual']==1]

In [0]:
df_regioes_concat.rename(columns={'SG_UF_x':'SG_UF'}, inplace=True)

In [0]:
df_regioes_concat.to_excel('teste.xlsx')

In [0]:
df_regioes_concat.head()

In [0]:
resumo1 = df_regioes_concat[['SG_UF', 'NM_MUNICIPIO', 'NM_REGIAO_SAUDE_ATUAL']]

In [0]:
df_reg_mesmos_municipios = df_regioes_concat[df_regioes_concat['flag_atual']==0]

In [0]:
df_reg_mesmos_municipios.shape

In [0]:
municipios_sem_mudar_regiao = df_reg_mesmos_municipios.shape[0]

In [0]:
municipios_sem_mudar_regiao

In [0]:
df_reg_mesmos_municipios.groupby('NM_REGIAO_SAUDE_ATUAL').count()

In [0]:
df_reg_mesmos_municipios.groupby('NM_REGIAO_SAUDE_ANTERIOR').count()

In [0]:
resumo2 = df_reg_mesmos_municipios.groupby(['NM_REGIAO_SAUDE_ATUAL','NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF']).count()['CD_MUNICIPIO'].reset_index()

In [0]:
resumo2.sort_values('CD_MUNICIPIO', ascending=False, inplace=True)

In [0]:
resumo2

In [0]:
df_regioes_concat.sort_values('SG_UF', ascending=True, inplace=True)

In [0]:
for row in df_regioes_concat.loc[df_regioes_concat['lista_regiao_anterior'].isnull(), 'lista_regiao_anterior'].index:
    df_regioes_concat.at[row, 'lista_regiao_anterior'] = []

In [0]:
df_regioes_concat[df_regioes_concat['lista_regiao_anterior'].isnull()]

In [0]:
df_regioes_concat['flag_municipio_entrou'] = df_regioes_concat['lista_regiao_atual'].map(set) - df_regioes_concat['lista_regiao_anterior'].map(set)

In [0]:
df_regioes_concat['flag_municipio_saiu'] = df_regioes_concat['lista_regiao_anterior'].map(set) - df_regioes_concat['lista_regiao_atual'].map(set)

In [0]:
df_regioes_concat[df_regioes_concat['NM_REGIAO_SAUDE_ATUAL']=='Baixo Acre e Purus']

In [0]:
df_regioes_concat[df_regioes_concat['NM_REGIAO_SAUDE_ATUAL']=='Norte']

In [0]:
df_regioes_concat['cont_municipio_entrou'] = [len(x) for x in df_regioes_concat['flag_municipio_entrou']]

In [0]:
df_regioes_concat['cont_municipio_saiu'] = [len(x) for x in df_regioes_concat['flag_municipio_saiu']]

In [0]:
df_regioes_concat[(df_regioes_concat['flag_atual']==0)&(df_regioes_concat['cont_municipio_entrou']>1)]

In [0]:
df_regioes_concat[(df_regioes_concat['flag_atual']==0)&(df_regioes_concat['cont_municipio_saiu']>1)]

In [0]:
df_estatistica_entrada = df_regioes_concat[(df_regioes_concat['flag_atual']==1)&(df_regioes_concat['cont_municipio_entrou']!=0)]

In [0]:
df_estatistica_entrada.shape

In [0]:
df_result_atual = df_estatistica_entrada.groupby(['SG_UF','NM_REGIAO_SAUDE_ATUAL'])['cont_municipio_entrou', 'cont_municipio_saiu'].agg(['count','min','max','mean']).reset_index()

In [0]:
df_result_atual

In [0]:
df_result_anterior = df_estatistica_entrada.groupby(['SG_UF','NM_REGIAO_SAUDE_ANTERIOR'])['cont_municipio_entrou', 'cont_municipio_saiu'].agg(['count','min','max','mean']).reset_index()

In [0]:
df_result_anterior.head()

In [0]:
df_result_atual.to_excel('resultado_atual.xlsx')

In [0]:
df_result_anterior.to_excel('resultado_anterior.xlsx')

In [0]:
df_estatistica_entrada.to_excel('resumo_geral_movimentacao.xlsx')

In [0]:
# Sum list of values within a pandas df
# df = [sum(a) for a in zip(*df['Val'])]